### Prpmpt

In [ ]:
from llama_index.core import PromptTemplate

template = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this information, please answer the question: {query_str}\n"
)
qa_template = PromptTemplate(template)

# you can create text prompt (for completion API)
prompt = qa_template.format(context_str=..., query_str=...)

## Dataset

Amod/mental_health_counseling_conversations

In [4]:
import datasets

dataset = datasets.load_dataset("Amod/mental_health_counseling_conversations")

print(dataset)

c:\Users\h4has\anaconda3\envs\rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['Context', 'Response'],
        num_rows: 3512
    })
})


In [5]:
print(dataset['train'])
print(type(dataset['train']))

Dataset({
    features: ['Context', 'Response'],
    num_rows: 3512
})
<class 'datasets.arrow_dataset.Dataset'>


In [6]:
data_dict = [data for data in dataset['train']]

In [7]:
print(len(data_dict))

3512


In [2]:
for data in dataset['train']:
    print(data)
    break

{'Context': "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?", 'Response': "If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media. \xa0Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is someho

## Model

In [27]:
# model = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
model = 't5-small'

In [26]:
# from transformers import LlamaTokenizer, Llama

# tokenizer = LlamaTokenizer.from_pretrained(model)
# model = LlamaForCausalLM.from_pretrained(model)


In [28]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [29]:
# query = "What is mental health?"
# inputs = tokenizer(query, return_tensors="pt")
# print(inputs)

In [6]:

outputs = model.generate(**inputs)
print(outputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

tensor([[   0, 2751,  229, 2550,  533,   58,    1]])
Was ist mental health?


c:\Users\h4has\anaconda3\envs\rag\lib\site-packages\transformers\generation\utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


## Basic Pipeline

In [12]:
import pandas as pd

In [13]:
df = pd.DataFrame(data_dict)

In [14]:
df.head(3)

,Context,Response
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb..."
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see..."
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...


In [17]:
df.to_csv('data.csv', index=False)

In [8]:
# Retreiving
# Indexing - Elasticsearch, TF-IDF 
# Querying

In [9]:
import minisearch


ms = minisearch.Index(
    text_fields=["Response"],
    keyword_fields=["Context"],
)

In [10]:
ms

create index - minsearch

In [11]:
ms.fit(data_dict)

search using index

In [21]:
query = "It is very scary for me to socialize with unknown people"

In [41]:
boost = {'question': 3.0, 'section': 0.5}

results = ms.search(
    query=query,
    # filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=3
)

In [42]:
results

[{'Context': "Sometimes I can't stop thinking about life after death. I was raised in a religion that teaches that we will live on forever either in hell or in heaven. \r\nWhen I think of living forever (even if it is in heaven which should be good), I feel overwhelmed. I don't like the thought of living forever and ever and ever. Sometimes I just can't get the thought out of my mind and the thoughts lead to panic and anxiety. \r\nAm I crazy? I don't think these thoughts are normal.",
  'Response': "You might be surprised how normal you are. Anxiety is incredibly common and while your particular type of existential anxiety might be unique to you - it is very difficult for most people to really comprehend what happens after we die - regardless of the religious or philosophical belief systems we hold. It is the ultimate unknown and some philosophers and psychologists believe that at the root of our day-to-day anxieties is the fear of death or fear of the unknown. \xa0Just as it can be re

In [30]:
prompt_template = """
You're a mental health assistant. Provide friendly answers based on the context.
QUESTION: {question}

CONTEXT: 
{response}
""".strip()

In [ ]:
prompt = prompt_template.format(question=query, response=results[0]['Response'])